# Capstone 1 (from Chapter 5) — Monte Carlo Estimation of Pi

In [1]:
!uv pip install numpy

Audited 1 package in 0.55ms


In [2]:
LOGGING_ENABLED = True

In [3]:
import logging

from llm_agents_from_scratch.logger import enable_console_logging

if LOGGING_ENABLED:
    enable_console_logging(logging.INFO)

## Build Tools

In [4]:
from llm_agents_from_scratch.tools import PydanticFunctionTool

### Tool: `generate_random_sample`

In [5]:
import uuid

import numpy as np
from pydantic import BaseModel, Field

# Global registry to store samples
SAMPLE_REGISTRY: dict[str, list[tuple[float, float]]] = {}


class RandomSampleParams(BaseModel):
    """Params for generate_random_sample."""

    n: int = Field(description="The number of random points to generate")


class RandomSampleResult(BaseModel):
    """Result from generate_random_sample."""

    sample_id: str = Field(
        description="Pass this sample_id to monte_carlo_estimate",
    )
    n: int = Field(description="Number of points generated")


def generate_random_sample(params: RandomSampleParams) -> RandomSampleResult:
    """Generate n random points in [-1, 1] × [-1, 1].

    Returns a sample_id. Pass this sample_id directly to monte_carlo_estimate.
    """
    orig_pts = np.random.uniform(size=(params.n, 2))
    transformed = 1 - 2 * (1 - orig_pts)

    sample_id = str(uuid.uuid4())
    SAMPLE_REGISTRY[sample_id] = [tuple(pt) for pt in transformed.tolist()]

    return RandomSampleResult(sample_id=sample_id, n=params.n)


class MonteCarloEstimateParams(BaseModel):
    """Params for monte_carlo_estimate."""

    sample_id: str = Field(
        description="The sample_id returned by generate_random_sample",
    )


def monte_carlo_estimate(params: MonteCarloEstimateParams) -> float:
    """Estimate pi using Monte Carlo method.

    Args:
        params: Contains sample_id from generate_random_sample.

    Returns:
        Estimate of pi (float).
    """
    points = SAMPLE_REGISTRY[params.sample_id]
    n = len(points)
    inside = sum((x**2 + y**2) < 1 for x, y in points)
    return (inside / n) * 4


random_sample_tool = PydanticFunctionTool(generate_random_sample)
monte_carlo_estimate_tool = PydanticFunctionTool(monte_carlo_estimate)

In [6]:
rs = generate_random_sample(RandomSampleParams(n=1000))
str(rs)

"sample_id='c3ca96ce-2f01-4ed6-a185-2f50fef84a94' n=1000"

In [7]:
pi_estimate = monte_carlo_estimate(
    MonteCarloEstimateParams(sample_id=rs.sample_id),
)
pi_estimate

3.208

In [8]:
monte_carlo_estimate_tool.parameters_json_schema

{'description': 'Params for monte_carlo_estimate.',
 'properties': {'sample_id': {'description': 'The sample_id returned by generate_random_sample',
   'title': 'Sample Id',
   'type': 'string'}},
 'required': ['sample_id'],
 'title': 'MonteCarloEstimateParams',
 'type': 'object'}

## Define our LLMAgent

In [9]:
from llm_agents_from_scratch import LLMAgent
from llm_agents_from_scratch.llms import OllamaLLM

model = "qwen3:8b"
llm = OllamaLLM(model)
llm_agent = LLMAgent(
    llm=llm,
    tools=[random_sample_tool, monte_carlo_estimate_tool],
)

## Define the Task

In [10]:
from llm_agents_from_scratch.data_structures import Task

In [11]:
instruction_template = """
You are given tools to estimate pi using Monte Carlo methods.

Your target: Estimate pi accurate to 4 decimal places (3.1415).

<tools>
1. `generate_random_sample(n)` → Returns a sample_id and n
2. `monte_carlo_estimate(sample_id)` → Returns a pi estimate (float)
</tools>

<workflow>
1. Call generate_random_sample(n=1000)
2. Call monte_carlo_estimate with the sample_id you received
3. Check if estimate is correct to four decimal places of PI
   i.e., 3.1415 and break
4. Repeat, don't stop until you hit the desired precision, increasing
   sample size of random if necessary
</workflow>

<warnings>
NEVER fabricate tool results—PI estimates should only come from
    monte_carlo_estimate tool.
NEVER make multiple tool calls in one response.
NEVER continue after a tool call - end your response immediately.
ALWAYS wait for the actual tool response before proceeding.
</warnings>
""".strip()

In [12]:
task = Task(
    instruction=instruction_template,
)

## Perform the Task

In [13]:
handler = llm_agent.run(task, max_steps=10)

INFO (llm_agents_fs.LLMAgent) :      🚀 Starting task: You are given tools to estimate pi using Monte Carlo methods.

Your target: Estimate pi accurate to 4 decimal places (3.1415).

<tool...[TRUNCATED]
INFO (llm_agents_fs.TaskHandler) :      ⚙️ Processing Step: You are given tools to estimate pi using Monte Carlo methods.

Your target: Estimate pi accurate to 4 decimal places (3.1415).

<t...[TRUNCATED]
INFO (llm_agents_fs.TaskHandler) :      🛠️ Executing Tool Call: generate_random_sample
INFO (llm_agents_fs.TaskHandler) :      ✅ Successful Tool Call: sample_id='5818206c-8391-4e2f-ad60-1230e3df3087' n=1000
INFO (llm_agents_fs.TaskHandler) :      ✅ Step Result: I need to call the monte_carlo_estimate tool with the sample_id '5818206c-8391-4e2f-ad60-1230e3df3087' to get the pi estimate.
INFO (llm_agents_fs.TaskHandler) :      🧠 New Step: Check the returned pi estimate for accuracy to four decimal places (3.1415). If not accurate, generate a new sample with a larger n and re...[TRUNCATED]

In [14]:
handler.done()

False

In [17]:
print(handler.rollout)

=== Task Step Start ===

💬 assistant: The current instruction is 'You are given tools to estimate pi using Monte Carlo methods.

Your target: Estimate pi accurate to 4 decimal places (3.1415).

<tools>
1. `generate_random_sample(n)` → Returns a sample_id and n
2. `monte_carlo_estimate(sample_id)` → Returns a pi estimate (float)
</tools>

<workflow>
1. Call generate_random_sample(n=1000)
2. Call monte_carlo_estimate with the sample_id you received
3. Check if estimate is correct to four decimal places of PI
   i.e., 3.1415 and break
4. Repeat, don't stop until you hit the desired precision, increasing
   sample size of random if necessary
</workflow>

<warnings>
NEVER fabricate tool results—PI estimates should only come from monte_carlo_estimate tool.
NEVER make multiple tool calls in one response.
NEVER continue after a tool call - end your response immediately.
ALWAYS wait for the actual tool response before proceeding.
</warnings>'

💬 assistant: I need to make the following tool call

In [18]:
result = (
    handler.result() if not handler.exception() else str(handler.exception())
)
print(result)

Max steps reached.
